This Database holds information about a music store. For this project, you will be assisting the Chinook team with understanding the media in their store, their customers and employees, and their invoice information. To assist you in the queries ahead, the schema for the Database is provided below. You can see the columns that link tables together via the arrows.

In [29]:
# pip install sqlalchemy

In [30]:
# pip install ipython-sql

In [10]:
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt

conn = sqlite3.connect("chinook.sqlite")

In [23]:
sqlalchemy.create_engine("sqlite:///chinook.sqlite")

Engine(sqlite:///chinook.sqlite)

In [31]:
%load_ext sql

In [36]:
%sql sqlite:///chinook.sqlite

This Database holds information about a music store. For this project, you will be assisting the Chinook team with understanding the media in their store, their customers and employees, and their invoice information. To assist you in the queries ahead, the schema for the Database is provided below. You can see the columns that link tables together via the arrows.

veamos las tablas que tenemos en el dataset:


In [38]:
%%sql

SELECT name
FROM sqlite_master 
WHERE type='table'

 * sqlite:///chinook.sqlite
Done.


name
Album
Artist
Customer
Employee
Genre
Invoice
InvoiceLine
MediaType
Playlist
PlaylistTrack


## Tarea 1:

Which countries have the most Invoices? <br>
Use the Invoice table to determine the countries that have the most invoices.<br>
Provide a table of BillingCountry and Invoices ordered by the number of invoices for each country. <br>
The country with the most invoices should appear first.

### Which countries have the most Invoices? Use the Invoice table to determine the countries that have the most invoices.

In [50]:
%%sql

SELECT *
FROM INVOICE
LIMIT 1;

 * sqlite:///chinook.sqlite
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2007-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98


### Provide a table of BillingCountry and Invoices ordered by the number of invoices for each country. The country with the most invoices should appear first.

In [51]:
%%sql

SELECT BILLINGCOUNTRY,
COUNT(DISTINCT(INVOICEID)) as count_invoices
FROM INVOICE
GROUP BY BILLINGCOUNTRY
ORDER BY count_invoices DESC;


 * sqlite:///chinook.sqlite
Done.


BillingCountry,count_invoices
USA,91
Canada,56
France,35
Brazil,35
Germany,28
United Kingdom,21
Portugal,14
Czech Republic,14
India,13
Sweden,7


## Tarea 2:

Which city has the best customers? We would like to throw a promotional Music Festival in the city we made the most money.<br>Write a query that returns the 1 city that has the highest sum of invoice totals.<br>Return both the city name and the sum of all invoice totals.

##### CON ESTO VEMOS QUÉ COLUMNAS HAY EN INVOICE Y SI PODEMOS RELAZIONAR ALGUNA DE ESTAS CON ALGUNA DE CUSTOMER.

In [87]:
%%sql

SELECT *
FROM INVOICE
LIMIT 1;

 * sqlite:///chinook.sqlite
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2007-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98


VEMOS QUE HAY UN BILLINGCITY, QUE NOS DA LAS CIUDADES. VAMOS A USARLA Y SUMAR EL TOTAL DE CADA CIUDAD

In [76]:
%%sql

SELECT BILLINGCITY,
ROUND(SUM(TOTAL),2) AS TOTAL_SUM
FROM INVOICE
GROUP BY BILLINGCITY
ORDER BY TOTAL_SUM DESC
LIMIT 5;

 * sqlite:///chinook.sqlite
Done.


BillingCity,TOTAL_SUM
Prague,90.24
Paris,77.24
Mountain View,77.24
São Paulo,75.24
London,75.24


## Tarea 3.
Who is the best customer? The customer who has spent the most money will be declared the best customer.<BR> Build a query that returns the person who has spent the most money.<BR>I found the solution by linking the following three: Invoice, InvoiceLine, and Customer tables to retrieve this information, but you can probably do it with fewer!

##### VEMOS QUE EN CUSTOMER E INVOICE TENEMOS CUSTOMERID, VAMOS A HACER UN JOIN PARA JUNTAR TABLAS CON ESTE ENLACE:

In [84]:
%%sql

SELECT
        C.CUSTOMERID,
        ROUND(SUM(I.TOTAL),2) AS TOTAL
    FROM CUSTOMER AS C
    LEFT JOIN INVOICE AS I
    ON C.CUSTOMERID = I.CUSTOMERID
    GROUP BY C.CUSTOMERID
    ORDER BY TOTAL DESC
    LIMIT 1;

 * sqlite:///chinook.sqlite
Done.


CustomerId,TOTAL
6,49.62


## Tarea 4:
Use your query to return the email, first name, last name, and Genre of all Rock Music listeners. Return your list ordered alphabetically by email address starting with A

In [106]:
%%sql

SELECT DISTINCT(C.EMAIL), C.FIRSTNAME, C.LASTNAME, G.NAME
    FROM GENRE AS G
    JOIN TRACK AS T
    ON G.GENREID = T.GENREID
    JOIN INVOICELINE AS IL
    ON T.TRACKID = IL.TRACKID
    JOIN INVOICE AS I
    ON IL.INVOICEID = I.INVOICEID
    JOIN CUSTOMER AS C
    ON I.CUSTOMERID = C.CUSTOMERID
    WHERE UPPER(G.NAME) = 'ROCK'
    ORDER BY C.EMAIL ASC
    LIMIT 5;

 * sqlite:///chinook.sqlite
Done.


Email,FirstName,LastName,Name
aaronmitchell@yahoo.ca,Aaron,Mitchell,Rock
alero@uol.com.br,Alexandre,Rocha,Rock
astrid.gruber@apple.at,Astrid,Gruber,Rock
bjorn.hansen@yahoo.no,Bjørn,Hansen,Rock
camille.bernard@yahoo.fr,Camille,Bernard,Rock


### Tarea 5:

Who is writing the rock music? Now that we know that our customers love rock music, we can decide which musicians to invite to play at the concert. Let’s invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name and total track count of the top 10 rock bands.

In [146]:
%%sql

SELECT A.NAME, COUNT (G.NAME) AS G_COUNT
    FROM ARTIST AS A, GENRE AS G
    JOIN ALBUM AS AL
    ON A.ARTISTID = AL.ARTISTID
    JOIN TRACK AS T
    ON AL.ALBUMID = T.ALBUMID
    WHERE UPPER(G.NAME) = 'ROCK'
    ORDER BY G_COUNT ASC
    LIMIT 5;

 * sqlite:///chinook.sqlite
Done.


Name,G_COUNT
AC/DC,3503


In [145]:
%%sql

SELECT A.NAME, COUNT(G.NAME) AS G_COUNT
    FROM ARTIST AS A
    JOIN ALBUM AS AL
    ON A.ARTISTID = AL.ARTISTID
    JOIN TRACK AS T
    ON AL.ALBUMID = T.ALBUMID
    JOIN GENRE AS G
    ON T.GENREID = G.GENREID
    WHERE UPPER(G.NAME) = 'ROCK'
    ORDER BY G_COUNT ASC
    LIMIT 10;

 * sqlite:///chinook.sqlite
Done.


Name,G_COUNT
AC/DC,1297


In [144]:
%%sql

SELECT A.NAME, COUNT(G.NAME) AS G_COUNT
    FROM TRACK AS T
    JOIN GENRE AS G
    ON T.GENREID = G.GENREID
    JOIN ALBUM AS AL
    ON AL.ALBUMID = T.ALBUMID
    JOIN ARTIST AS A
    ON AL.ARTISTID = A.ARTISTID
    WHERE UPPER(G.NAME) = 'ROCK'
    ORDER BY G_COUNT ASC
    LIMIT 10;

 * sqlite:///chinook.sqlite
Done.


Name,G_COUNT
AC/DC,1297
